#Librerias

In [ ]:
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [ ]:
drive.mount('/content/drive')
df = pd.concat(
    pd.read_excel("/content/drive/MyDrive/Colab Notebooks/online_retail_II.xlsx",
                 sheet_name=['Year 2009-2010', 'Year 2010-2011']),
    ignore_index=True
)

Mounted at /content/drive


#Limpieza

In [ ]:
def eliminar_duplicados_personalizados(df, columnas_clave=None, considerar_fecha=True, verbose=True):
    """
    Identifica y elimina duplicados
    """
    columnas_clave = ['Invoice', 'StockCode', 'Quantity','Price','InvoiceDate']
    # Contar duplicados
    duplicados = df.duplicated(subset=columnas_clave, keep='first').sum()

    # Eliminar duplicados
    df_limpio = df.drop_duplicates(subset=columnas_clave, keep='first')

    # Mostrar información si verbose=True
    if verbose:
        print("\n=== Análisis de duplicados ===")
        print(f"Registros originales: {len(df)}")
        print(f"Registros duplicados: {duplicados}")
        print(f"Registros único: {len(df_limpio)}")
        print(f"Porcentaje de duplicados: {duplicados/len(df)*100:.2f}%")

    return df_limpio, duplicados

In [ ]:
df_limpio, num_duplicados = eliminar_duplicados_personalizados(df)


=== Análisis de duplicados ===
Registros originales: 1067371
Registros duplicados: 34337
Registros único: 1033034
Porcentaje de duplicados: 3.22%


In [ ]:
def filtrar_irrelevantes(df):
    """
    Elimina transacciones sin ID de cliente
    Elimina pedidos cancelados (facturas que comienzan con 'C')
    """
    df_filtrado = df.copy()
    # 1. Eliminar registros sin Customer ID
   # df_filtrado = df_filtrado.dropna(subset=['Customer ID'])

    # 2. Eliminar cancelaciones (facturas que empiezan con 'C')
    cancelaciones = df_filtrado[df_filtrado['Invoice'].astype(str).str.startswith('C')].index
    df_filtrado = df_filtrado.drop(cancelaciones)

    # 3. Eliminar precios negativos
    precios_negativos = df_filtrado[df_filtrado['Price'] < 0]
    df_filtrado = df_filtrado.drop(precios_negativos.index)

    precios_negativos = df_filtrado[df_filtrado['Quantity'] < 0]
    df_filtrado = df_filtrado.drop(precios_negativos.index)

    df_filtrado['Description'] = df_filtrado['Description'].fillna('Unspecified')
    df_filtrado['Customer ID'] = df_filtrado['Customer ID'].fillna(0)

    #print(f"- Sin Customer ID: {len(df) - len(df_filtrado) - df_filtrado['Invoice'].astype(str).str.startswith('C').sum()}")
    print(f"- Cancelaciones: {df_filtrado['Invoice'].astype(str).str.startswith('C').sum()}")
    print(f"- Precios negativos: {len(precios_negativos)}")
    print(f"- Cantidades negativas: {len(precios_negativos)}")
    print(f"Total final: {len(df_filtrado)} registros")
    return df_filtrado

In [ ]:
df_limpio = filtrar_irrelevantes(df_limpio)

- Cancelaciones: 0
- Precios negativos: 3393
- Cantidades negativas: 3393
Total final: 1010532 registros


In [ ]:
df_limpio['Invoice'] = df_limpio['Invoice'].astype('string')
df_limpio['StockCode'] = df_limpio['StockCode'].astype('string')
df_limpio['Country'] = df_limpio['Country'].astype('string')
df_limpio['Description'] = df_limpio['Description'].astype('string')
df_limpio['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [ ]:
df_limpio['TotalValue'] = df_limpio['Quantity'] * df_limpio['Price']
df_limpio['TotalValue']

,TotalValue
0,83.40
1,81.00
2,81.00
3,100.80
4,30.00
...,...
1067366,12.60
1067367,16.60
1067368,16.60
1067369,14.85


# Parte 3

In [ ]:
    df_limpio['year'] = df_limpio['InvoiceDate'].dt.year
    df_limpio['month'] = df_limpio['InvoiceDate'].dt.month
    df_limpio['day'] = df_limpio['InvoiceDate'].dt.day
    df_limpio['day_of_week'] = df_limpio['InvoiceDate'].dt.dayofweek  # 0=Lunes, 6=Domingo
    df_limpio['hour'] = df_limpio['InvoiceDate'].dt.hour
    df_limpio['YearMonth'] = df_limpio['InvoiceDate'].dt.to_period('M')
    df_limpio['Weekday'] = df_limpio['InvoiceDate'].dt.day_name()

In [ ]:
def assign_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'

df_limpio['season'] = df_limpio['month'].apply(assign_season)


In [ ]:
def time_of_day(hour):
    if 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    elif 18 <= hour < 24:
        return 'Evening'
    else:
        return 'Night'

df_limpio['time_of_day'] = df_limpio['hour'].apply(time_of_day)


In [ ]:
df_limpio['Country'].unique()

<StringArray>
[      'United Kingdom',               'France',                  'USA',
              'Belgium',            'Australia',                 'EIRE',
              'Germany',             'Portugal',              'Denmark',
          'Netherlands',               'Poland',      'Channel Islands',
                'Spain',               'Cyprus',               'Greece',
               'Norway',              'Austria',               'Sweden',
 'United Arab Emirates',              'Finland',                'Italy',
          'Switzerland',                'Japan',          'Unspecified',
              'Nigeria',                'Malta',              'Bahrain',
                  'RSA',              'Bermuda',            'Hong Kong',
            'Singapore',             'Thailand',               'Israel',
            'Lithuania',          'West Indies',              'Lebanon',
                'Korea',               'Brazil',               'Canada',
              'Iceland',         'Sau

In [ ]:
europa_oeste = ['United Kingdom', 'France', 'Germany', 'Netherlands', 'Austria', 'Switzerland', 'Belgium', 'Ireland', 'Portugal', 'Spain', 'Italy', 'Sweden', 'Denmark', 'Finland']
europa_este = ['Poland', 'Lithuania', 'Czech Republic']
asia = ['Japan', 'Hong Kong', 'Singapore', 'Thailand', 'Korea']
medio_oriente = ['United Arab Emirates', 'Bahrain', 'Israel', 'Saudi Arabia']
africa = ['Nigeria', 'RSA']
america = ['USA', 'Canada', 'Brazil', 'Bermuda']
otros = ['Cyprus', 'Malta', 'Iceland', 'Channel Islands', 'West Indies', 'Lebanon', 'European Community']
sin_especificar = ['Unspecified']

def categorizar_pais(pais):
    if pais in europa_oeste:
        return 'Western Europe'
    elif pais in europa_este:
        return 'Eastern Europe'
    elif pais in asia:
        return 'Asia'
    elif pais in medio_oriente:
        return 'Middle East'
    elif pais in africa:
        return 'Africa'
    elif pais in america:
        return 'Americas'
    elif pais in sin_especificar:
        return 'Unspecified'
    else:
        return 'Other'

df_limpio['Country_cat'] = df_limpio['Country'].apply(categorizar_pais)


In [ ]:
top_products = df_limpio['StockCode'].value_counts().nlargest(20).index
df_limpio['is_top_product'] = df_limpio['StockCode'].isin(top_products).astype(int)
df_limpio['season'] = df_limpio['season'].astype('string')  # Correcto
df_limpio['time_of_day'] = df_limpio['time_of_day'].astype('string')  # Correcto

In [ ]:
df_codificado = df_limpio.copy()

columnas_a_codificar = ['Country_cat', 'season', 'time_of_day']

# Aplicar OneHotEncoder
encoder = OneHotEncoder(sparse_output=True, handle_unknown='ignore')
X_encoded = encoder.fit_transform(df_codificado[columnas_a_codificar])

nombres_columnas = encoder.get_feature_names_out(columnas_a_codificar)
df_encoded = pd.DataFrame.sparse.from_spmatrix(X_encoded, columns=nombres_columnas, index=df_codificado.index)

# Concatenar con el DataFrame original
df_codificado = pd.concat([df_codificado, df_encoded], axis=1)
print(df_codificado.head())

  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

          InvoiceDate  Price  Customer ID         Country  TotalValue  year  \
0 2009-12-01 07:45:00   6.95      13085.0  United Kingdom        83.4  2009   
1 2009-12-01 07:45:00   6.75      13085.0  United Kingdom        81.0  2009   
2 2009-12-01 07:45:00   6.75      13085.0  United Kingdom        81.0  2009   
3 2009-12-01 07:45:00   2.10      13085.0  United Kingdom       100.8  2009   
4 2009-12-01 07:45:00   1.25      13085.0  United Kingdom        30.0  2009   

   ...  Country_cat_Other  Country_cat_Unspecified  \
0  ...                  0                        0

In [ ]:
df_codificado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1010532 entries, 0 to 1067370
Data columns (total 35 columns):
 #   Column                      Non-Null Count    Dtype             
---  ------                      --------------    -----             
 0   Invoice                     1010532 non-null  string            
 1   StockCode                   1010532 non-null  string            
 2   Description                 1010532 non-null  string            
 3   Quantity                    1010532 non-null  int64             
 4   InvoiceDate                 1010532 non-null  datetime64[ns]    
 5   Price                       1010532 non-null  float64           
 6   Customer ID                 1010532 non-null  float64           
 7   Country                     1010532 non-null  string            
 8   TotalValue                  1010532 non-null  float64           
 9   year                        1010532 non-null  int32             
 10  month                       1010532 non-null  i

<ipython-input-18-be1a90608350>:1: FutureWarning: Allowing arbitrary scalar fill_value in SparseDtype is deprecated. In a future version, the fill_value must be a valid value for the SparseDtype.subtype.
  df_codificado.info()


El RandomForestRegressor es un algoritmo de aprendizaje supervisado diseñado para problemas de regresión, es decir, para predecir valores numéricos

In [ ]:
# Importaciones necesarias
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd

#Definir variables
y = df_codificado['TotalValue']
X = df_codificado.drop(columns=['StockCode', 'InvoiceDate', 'Invoice', 'Country_cat', 'Country',
                              'Description', 'YearMonth', 'Weekday', 'TotalValue', 'season',
                              'time_of_day', 'day_of_week', 'Customer ID', 'day', 'hour',
                              'is_top_product'])

#Preprocesamiento (transformación logarítmica)
X['Quantity'] = np.log1p(X['Quantity'])
X['Price'] = np.log1p(X['Price'])

#División de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Modelo
model = RandomForestRegressor(
    n_estimators=100,      # Número de árboles en el bosque
    max_depth=10,          # Profundidad máxima de cada árbol
    min_samples_split=5,   # Mínimo número de muestras para dividir un nodo
    min_samples_leaf=2,    # Mínimo número de muestras en hojas
    max_features='sqrt',   # Número de features a considerar
    random_state=42,       # Semilla para reproducibilidad
    n_jobs=-1,             # Usa todos los núcleos del CPU
    verbose=1              # Muestra progreso durante el entrenamiento
)

model.fit(X_train, y_train)

# 5. Evaluar el modelo
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("\n" + "="*50)
print(f"🚀 RMSE: {rmse:.2f}")
print(f"📊 R² Score: {r2:.3f}")
print("="*50 + "\n")
importances = model.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
}).sort_values('Importance', ascending=False)

print("🔍 Importancia de características:")
print(feature_importance_df.to_string(index=False))

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   35.7s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.1min finished
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.5s



🚀 RMSE: 40.74
📊 R² Score: 0.695

🔍 Importancia de características:
                   Feature   Importance
                  Quantity 7.514133e-01
                     Price 2.311772e-01
                     month 4.620618e-03
             season_Summer 3.244788e-03
                      year 2.172356e-03
     time_of_day_Afternoon 1.939311e-03
             season_Autumn 1.418612e-03
             season_Winter 1.065421e-03
       time_of_day_Morning 7.883532e-04
             season_Spring 7.637109e-04
Country_cat_Western Europe 6.496081e-04
         Country_cat_Other 3.476551e-04
          Country_cat_Asia 1.908993e-04
       time_of_day_Evening 1.894751e-04
   Country_cat_Unspecified 6.583556e-06
   Country_cat_Middle East 5.217794e-06
      Country_cat_Americas 4.264106e-06
Country_cat_Eastern Europe 2.078758e-06
        Country_cat_Africa 5.442174e-07


[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    1.0s finished


* El error cuadrático medio es de 40.74 unidades en las predicciones.Por ejemplo, al predecir en valor total el error promedio sería de 40.74 libras esterlinas.
* El modelo explica 69.5% de la variabilidad de los datos esto sugiere que hay un 30.5% de variabilidad no capturada (quizás por variables no incluidas).
* Quantity y Price son las variables que dan más información.
